# **Ocular Disease Recognition project**

# Thông tin về dữ liệu này

Nhận dạng thông minh bệnh mắt (ODIR) là cơ sở dữ liệu nhãn khoa có cấu trúc gồm 5.000 bệnh nhân theo độ tuổi, ảnh đáy mắt màu từ mắt trái và mắt phải và từ khóa chẩn đoán của bác sĩ từ bác sĩ. Tập dữ liệu này đại diện cho tập hợp bệnh nhân ''đời thực'' thông tin được Công ty TNHH Công nghệ Y tế Shanggong thu thập từ các bệnh viện/trung tâm y tế khác nhau ở Trung Quốc. Trong các tổ chức này, hình ảnh đáy mắt được chụp bằng nhiều máy ảnh khác nhau trên thị trường, chẳng hạn như Canon, Zeiss và Kowa, dẫn đến độ phân giải hình ảnh khác nhau. Các chú thích được gắn nhãn bởi những người đọc đã qua đào tạo với quản lý kiểm soát chất lượng. Họ phân loại bệnh nhân thành 8 nhãn bao gồm:

*   Normal (N) : Bình thường
*   Diabetthường: Bệnh tiểu đường
*   Glaucoma (G): Bệnh tăng nhãn áp
*   Cataract (C): Đục thủy tinh thể
*   Age related Macular Degeneration (A): Thoái hóa điểm vàng liên quan đến tuổi tác
*   Hypertension (H): Tăng huyết áp
*   Pathological Myopia (M): Cận thị bệnh lý
*   Other diseases (O): Các bệnh/bất thường khác

In [1]:
!pip install plotly

# Nhập các thư viện cần thiết

In [ ]:
# Các thư viện chính
import numpy as np  # Thư viện cho các phép toán số học và mảng n-dimensional
import pandas as pd  # Thư viện cho xử lý và phân tích dữ liệu dạng bảng
import os  # Thư viện cho tương tác với hệ thống tệp
import cv2  # Thư viện cho xử lý hình ảnh và video
from collections import Counter  # Thư viện cho đếm tần suất các phần tử trong một danh sách

# Thư viện trực quan hóa
import plotly.express as px  # Thư viện cho trực quan hóa dữ liệu đơn giản và nhanh chóng
import plotly.figure_factory as ff  # Thư viện cho tạo các hình vẽ phức tạp hơn
import plotly.graph_objects as go  # Thư viện cho tạo các đối tượng đồ họa tùy chỉnh
import matplotlib.pyplot as plt  # Thư viện cho vẽ biểu đồ 2D
import seaborn as sns  # Thư viện cho trực quan hóa dữ liệu thống kê, cải tiến từ matplotlib
from mpl_toolkits.axes_grid1 import ImageGrid  # Công cụ để tạo lưới hình ảnh trong matplotlib

# Các công cụ máy học và tiền xử lý
from sklearn.cluster import KMeans  # Nhập mô hình KMeans cho phân cụm dữ liệu
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV  
# Nhập các công cụ để chia tập dữ liệu, thực hiện phân lớp có stratification và tìm kiếm siêu tham số qua Grid Search

from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, classification_report, confusion_matrix, multilabel_confusion_matrix  
# Nhập các chỉ số đánh giá hiệu suất mô hình như độ chính xác, F1-score, AUC-ROC, và các báo cáo phân loại

from sklearn.utils.class_weight import compute_class_weight  # Tính toán trọng số cho các lớp không cân bằng trong tập dữ liệu

from sklearn.preprocessing import StandardScaler, LabelEncoder  
# Nhập các công cụ tiền xử lý: chuẩn hóa dữ liệu và mã hóa nhãn cho các biến phân loại

from sklearn.pipeline import Pipeline  # Nhập lớp Pipeline để xây dựng quy trình xử lý dữ liệu và mô hình hóa

# Thư viện tensorflow và keras cho CNN
import tensorflow as tf  # Thư viện cho học sâu
from tensorflow.keras.models import Sequential, Model  # Nhập các lớp mô hình cho Keras
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, GlobalAveragePooling2D, BatchNormalization
# Nhập các lớp cho mạng nơ-ron tích chập (CNN)

from tensorflow.keras.optimizers import Adam, SGD, Nadam  # Nhập các bộ tối ưu hóa cho huấn luyện mô hình
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau  
# Nhập các callback để điều chỉnh quá trình huấn luyện mô hình

from tensorflow.keras.preprocessing.image import ImageDataGenerator  # Nhập công cụ để tạo dữ liệu hình ảnh cho huấn luyện
from tensorflow.keras.utils import to_categorical  # Chuyển đổi nhãn thành dạng nhị phân
from tensorflow.keras.applications import EfficientNetB0, ResNet50, DenseNet121, VGG16, InceptionV3  
# Nhập các mô hình CNN tiền huấn luyện để sử dụng cho việc phân loại

from tensorflow.keras import backend as K  # Nhập backend của Keras để thao tác với tensor

# Xử lý hình ảnh
from PIL import Image  # Thư viện cho xử lý hình ảnh
import albumentations as A  # Thư viện cho tăng cường dữ liệu hình ảnh

# Thư viện tiện ích
import time  # Thư viện để làm việc với thời gian
import logging  # Thư viện để ghi nhật ký
import gc  # Thư viện để quản lý bộ nhớ (garbage collection)

# Thư viện hệ thống
import sys  # Thư viện cho tương tác với hệ thống và môi trường thực thi

# Tải dữ liệu

In [ ]:
for dirname, _, filenames in os.walk('/kaggle/input'):
    print(dirname)

In [ ]:
df = pd.read_csv("/kaggle/input/nhn-bit-bnh-v-mt/Ocular Disease Recognition/full_df.csv")
df.head(4)

In [ ]:
plt.figure(figsize=(8,8))
plt.pie(df['labels'].value_counts(),autopct="%0.1F%%",
        labels=['Normal','Diabetes','Glaucoma','Cataract','Age related Macular Degeneration','Hypertension',
                'Pathological Myopia','Other diseases/abnormalities']
                ,shadow=True,explode=[0.2,0.09,0,0,0,0,0,0]
                )

In [ ]:
df = pd.read_csv("/kaggle/input/nhn-bit-bnh-v-mt/Ocular Disease Recognition/full_df.csv")
df.count()

In [ ]:
# Đường dẫn đến thư mục chính chứa dữ liệu ODIR-5K
nested_odir_path = '/kaggle/input/nhn-bit-bnh-v-mt/Ocular Disease Recognition/ODIR-5K/ODIR-5K'

# Liệt kê các tệp trong thư mục chính
main_files = os.listdir(nested_odir_path)
print("Nội dung thư mục chính ODIR-5K:")
print(main_files)

# Liệt kê các tệp trong thư mục hình ảnh kiểm tra và hình ảnh huấn luyện
testing_images_path = os.path.join(nested_odir_path, 'Testing Images')  # Đường dẫn đến thư mục hình ảnh kiểm tra
training_images_path = os.path.join(nested_odir_path, 'Training Images')  # Đường dẫn đến thư mục hình ảnh huấn luyện

print("\nNội dung thư mục hình ảnh kiểm tra:")
print(os.listdir(testing_images_path)[:10])  # Hiển thị 10 tệp đầu tiên trong thư mục hình ảnh kiểm tra

print("\nNội dung thư mục hình ảnh huấn luyện:")
print(os.listdir(training_images_path)[:10])  # Hiển thị 10 tệp đầu tiên trong thư mục hình ảnh huấn luyện

In [ ]:
# Đường dẫn đến thư mục chứa hình ảnh đã được tiền xử lý
preprocessed_path = "/kaggle/input/nhn-bit-bnh-v-mt/Ocular Disease Recognition/preprocessed_images"

# Liệt kê các tệp trong thư mục hình ảnh đã được tiền xử lý
preprocessed_files = os.listdir(preprocessed_path)
print(f"Số lượng tệp trong thư mục preprocessed_images: {len(preprocessed_files)}")

# Hiển thị 10 tệp đầu tiên
print("\n10 tệp đầu tiên từ thư mục preprocessed_images:")
print(preprocessed_files[:10])  # Hiển thị 10 tệp đầu tiên

# Hiển thị 10 tệp cuối cùng
print("\n10 tệp cuối cùng từ thư mục preprocessed_images:")
print(preprocessed_files[-10:])  # Hiển thị 10 tệp cuối cùng

In [ ]:
def get_image_file_path(directory): # Hàm lấy đường dẫn file ảnh
    # Duyệt qua tất cả các tệp trong thư mục
    for file_name in os.listdir(directory):
        file_path = os.path.join(directory, file_name)  # Tạo đường dẫn đầy đủ cho tệp
        if os.path.isfile(file_path):  # Kiểm tra xem đó có phải là tệp không
            return file_path  # Trả về đường dẫn của tệp đầu tiên tìm thấy
    return None  # Nếu không tìm thấy tệp hợp lệ, trả về None

In [ ]:
# Lấy đường dẫn của một tệp hình ảnh hợp lệ từ thư mục hình ảnh kiểm tra
sample_testing_image_path = get_image_file_path(testing_images_path)

# Kiểm tra xem có tệp hình ảnh nào tìm thấy không
if sample_testing_image_path:
    # Mở hình ảnh
    sample_testing_image = Image.open(sample_testing_image_path)

    # Hiển thị hình ảnh
    plt.figure(figsize=(6, 6))
    plt.imshow(sample_testing_image)  # Hiển thị hình ảnh
    plt.title(f"Hình ảnh mẫu từ thư mục kiểm tra: {os.path.basename(sample_testing_image_path)}")  # Tiêu đề với tên tệp
    plt.axis('off')  # Tắt trục
    plt.show()  # Hiển thị hình ảnh
else:
    print("Không tìm thấy tệp hình ảnh hợp lệ trong thư mục testing_images")  # Thông báo nếu không có tệp nào

In [ ]:
# Lấy đường dẫn của một tệp hình ảnh hợp lệ từ thư mục hình ảnh huấn luyện
sample_training_image_path = get_image_file_path(training_images_path)

# Kiểm tra xem có tệp hình ảnh nào tìm thấy không
if sample_training_image_path:
    # Mở hình ảnh
    sample_training_image = Image.open(sample_training_image_path)

    # Hiển thị hình ảnh
    plt.figure(figsize=(6, 6))
    plt.imshow(sample_training_image)  # Hiển thị hình ảnh
    plt.title(f"Hình ảnh mẫu: {os.path.basename(sample_training_image_path)}")  # Tiêu đề với tên tệp
    plt.axis('off')  # Tắt trục
    plt.show()  # Hiển thị hình ảnh
else:
    print("Không tìm thấy tệp hình ảnh hợp lệ trong thư mục training_images")  # Thông báo nếu không có tệp nào

In [ ]:
# Hàm để hiển thị một hình ảnh từ thư mục đã cho
def display_sample_image(folder_path, file_name):
    file_path = os.path.join(folder_path, file_name)  # Tạo đường dẫn đầy đủ cho tệp
    image = Image.open(file_path)  # Mở hình ảnh

    # Hiển thị hình ảnh
    plt.figure(figsize=(6, 6))
    plt.imshow(image)
    plt.title(f"Hình ảnh mẫu: {file_name}")
    plt.axis('off')  # Tắt trục
    plt.show()  # Hiển thị hình ảnh

# Hiển thị một hình ảnh mẫu từ ODIR-5K
print("Hình ảnh từ ODIR-5K:")
sample_odir_image = os.listdir(testing_images_path)[0]  # Lấy tệp đầu tiên trong thư mục hình ảnh kiểm tra
display_sample_image(testing_images_path, sample_odir_image)  # Hiển thị hình ảnh ODIR-5K

# Hiển thị một hình ảnh mẫu từ thư mục hình ảnh đã được tiền xử lý
print("\nHình ảnh từ preprocessed_images:")
sample_preprocessed_image = os.listdir(preprocessed_path)[0]  # Lấy tệp đầu tiên trong thư mục hình ảnh đã được tiền xử lý
display_sample_image(preprocessed_path, sample_preprocessed_image)  # Hiển thị hình ảnh đã được tiền xử lý

In [ ]:
def has_cataract(text):
    if "cataract" in text:
        return 1
    else:
        return 0

In [ ]:
df["left_cataract"] = df["Left-Diagnostic Keywords"].apply(lambda x: has_cataract(x))
df["right_cataract"] = df["Right-Diagnostic Keywords"].apply(lambda x: has_cataract(x))

In [ ]:
left_cataract = df.loc[(df.C ==1) & (df.left_cataract == 1)]["Left-Fundus"].values
left_cataract[:15]

In [ ]:
right_cataract = df.loc[(df.C ==1) & (df.right_cataract == 1)]["Right-Fundus"].values
right_cataract[:15]

In [ ]:
print("Số lượng hình ảnh trong đục thủy tinh thể trái: {}".format(len(left_cataract)))
print("Số lượng hình ảnh trong đục thủy tinh thể phải: {}".format(len(right_cataract)))

In [ ]:
left_normal = df.loc[(df.C ==0) & (df["Left-Diagnostic Keywords"] == "normal fundus")]["Left-Fundus"].sample(250,random_state=42).values
right_normal = df.loc[(df.C ==0) & (df["Right-Diagnostic Keywords"] == "normal fundus")]["Right-Fundus"].sample(250,random_state=42).values
right_normal[:15]

In [ ]:
cataract = np.concatenate((left_cataract,right_cataract),axis=0)
normal = np.concatenate((left_normal,right_normal),axis=0)

In [ ]:
print(len(cataract),len(normal))

In [ ]:
import cv2
import random
from tqdm import tqdm
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import load_img,img_to_array
dataset_dir = "/kaggle/input/nhn-bit-bnh-v-mt/Ocular Disease Recognition/preprocessed_images"
image_size=224
labels = []
dataset = []
def create_dataset(image_category,label):
    for img in tqdm(image_category):
        image_path = os.path.join(dataset_dir,img)
        try:
            image = cv2.imread(image_path,cv2.IMREAD_COLOR)
            image = cv2.resize(image,(image_size,image_size))
        except:
            continue
        
        dataset.append([np.array(image),np.array(label)])
    random.shuffle(dataset)
    return dataset

In [ ]:
dataset = create_dataset(cataract,1)
len(dataset)

In [ ]:
dataset = create_dataset(normal,0)
len(dataset)

In [ ]:
plt.figure(figsize=(12,7))
for i in range(10):
    sample = random.choice(range(len(dataset)))
    image = dataset[sample][0]
    category = dataset[sample][1]
    if category== 0:
        label = "Normal"
    else:
        label = "Cataract"
    plt.subplot(2,5,i+1)
    plt.imshow(image)
    plt.xlabel(label)
plt.tight_layout()    

In [ ]:
x = np.array([i[0] for i in dataset]).reshape(-1,image_size,image_size,3)
y = np.array([i[1] for i in dataset])

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2)

# Phân tích và trực quan hóa dữ liệu

In [ ]:
# df.info(): Phương thức này cung cấp thông tin tổng quát về DataFrame, bao gồm:
# Số lượng hàng và cột.
# Tên cột và kiểu dữ liệu của từng cột.
# Số lượng giá trị không null trong mỗi cột.
# Hiển thị kiểu dữ liệu của DataFrame.
# Sử dụng bộ nhớ của DataFrame.
# Hiển thị thông tin về DataFrame

df.info()

In [ ]:
# isna(): Trả về một DataFrame mới với giá trị True cho các ô có giá trị 
# thiếu và False cho các ô có giá trị không thiếu.
# sum(): Khi được áp dụng trên DataFrame boolean (do isna() tạo ra), 
# nó sẽ tính tổng số lượng True (tương đương với số giá trị thiếu) cho mỗi cột.

df.isna().sum()

In [ ]:
# Thông tin df.describe() cung cấp:
# count: Số lượng giá trị không null.
# mean: Giá trị trung bình.
# std: Độ lệch chuẩn.
# min: Giá trị nhỏ nhất.
# 25%: Phân vị thứ 25.
# 50%: Phân vị thứ 50 (median).
# 75%: Phân vị thứ 75.
# max: Giá trị lớn nhất.
    
df.describe()

In [ ]:
# Phương thức df['Left-Diagnostic Keywords'].nunique() trong pandas được sử dụng 
# để đếm số lượng giá trị duy nhất (unique values) trong cột cụ thể của DataFrame. 
# Đây là cách kiểm tra sự đa dạng trong dữ liệu.

df['Left-Diagnostic Keywords'].nunique()

In [ ]:
# Phương thức df['Right-Diagnostic Keywords'].nunique() trong pandas được sử dụng 
# để đếm số lượng giá trị duy nhất trong cột Right-Diagnostic Keywords của DataFrame. 
# Đây là cách để kiểm tra sự đa dạng của các giá trị trong cột đó.

df['Right-Diagnostic Keywords'].nunique()

In [ ]:
# Phương thức df['Left-Diagnostic Keywords'].value_counts() trong pandas được sử dụng 
# để đếm số lần xuất hiện của mỗi giá trị trong cột Left-Diagnostic Keywords của DataFrame. 
# Đây là một công cụ hữu ích để phân tích dữ liệu và hiểu rõ hơn về phân phối các giá trị trong cột đó.

df['Left-Diagnostic Keywords'].value_counts()

In [ ]:
# Phương thức df['Right-Diagnostic Keywords'].value_counts() trong pandas được sử dụng
# để đếm số lần xuất hiện của mỗi giá trị trong cột Right-Diagnostic Keywords của DataFrame. 
# Đây là công cụ hữu ích để phân tích dữ liệu và hiểu rõ hơn về phân phối các giá trị trong cột đó.

df['Right-Diagnostic Keywords'].value_counts()

In [ ]:
# Tần suất bệnh lý ở mắt trái
# Tính số lần xuất hiện của mỗi chẩn đoán trong cột 'Left-Diagnostic Keywords'
left_diagnosis_counts = df['Left-Diagnostic Keywords'].value_counts()

# Tạo biểu đồ cột cho phân bố chẩn đoán mắt trái
fig_left = px.bar(
    x=left_diagnosis_counts.index,  # Các giá trị chẩn đoán
    y=left_diagnosis_counts.values,  # Số lượng tương ứng
    labels={'x': 'Chẩn đoán mắt trái', 'y': 'Số lượng'},  # Nhãn cho trục
    title='Phân bố các chẩn đoán mắt trái'  # Tiêu đề biểu đồ
)
fig_left.show()  # Hiển thị biểu đồ cho mắt trái

# Và đây là cho mắt phải
# Tính số lần xuất hiện của mỗi chẩn đoán trong cột 'Right-Diagnostic Keywords'
right_diagnosis_counts = df['Right-Diagnostic Keywords'].value_counts()

# Tạo biểu đồ cột cho phân bố chẩn đoán mắt phải
fig_right = px.bar(
    x=right_diagnosis_counts.index,  # Các giá trị chẩn đoán
    y=right_diagnosis_counts.values,  # Số lượng tương ứng
    labels={'x': 'Chẩn đoán mắt phải', 'y': 'Số lượng'},  # Nhãn cho trục
    title='Phân bố các chẩn đoán mắt phải'  # Tiêu đề biểu đồ
)
fig_right.show()  # Hiển thị biểu đồ cho mắt phải

# Ghi chú: Chẩn đoán bình thường có số lượng nhiều nhất trong phân bố của cả hai mắt

In [ ]:
# Tạo DataFrame để so sánh
df_comparison = df[['Left-Diagnostic Keywords', 'Right-Diagnostic Keywords']].copy()

# Thêm cột 'Both_Eyes_Same' để kiểm tra xem chẩn đoán hai mắt có giống nhau không
df_comparison['Both_Eyes_Same'] = df_comparison['Left-Diagnostic Keywords'] == df_comparison['Right-Diagnostic Keywords']

# Tính số lượng chẩn đoán giống nhau và khác nhau
comparison_counts = df_comparison['Both_Eyes_Same'].value_counts()

# Tạo biểu đồ cột để hiển thị số lượng chẩn đoán giống nhau so với khác nhau
fig_comparison = px.bar(
    x=comparison_counts.index,  # Trục x là giá trị True/False
    y=comparison_counts.values,  # Trục y là số lượng tương ứng
    labels={'x': 'Chẩn đoán giống nhau', 'y': 'Số lượng'},  # Nhãn cho trục
    title='Số lượng chẩn đoán giống nhau so với khác nhau (Mắt trái vs. Mắt phải)'  # Tiêu đề biểu đồ
)
fig_comparison.show()  # Hiển thị biểu đồ

# Ghi chú: True nếu chẩn đoán hai mắt giống nhau, False nếu không

In [ ]:
# So sánh 2 DataFrame
df_comparison

In [ ]:
from itertools import combinations
from collections import Counter

# Hàm này tìm mối tương quan chẩn đoán giữa mắt trái và mắt phải
def get_diagnosis_correlations(df):
    # Tạo danh sách các cặp chẩn đoán từ mắt trái và mắt phải
    left_right_pairs = list(zip(df['Left-Diagnostic Keywords'], df['Right-Diagnostic Keywords']))
    # Đếm số lần xuất hiện của mỗi cặp chẩn đoán
    pair_counts = Counter(left_right_pairs)
    return pair_counts.most_common()  # Trả về các cặp chẩn đoán phổ biến nhất

# Lấy các cặp chẩn đoán phổ biến nhất
diagnosis_pairs = get_diagnosis_correlations(df)

# Chuyển đổi dữ liệu để sử dụng với Plotly
labels, counts = zip(*diagnosis_pairs)  # Tách cặp chẩn đoán và số lượng
left_labels, right_labels = zip(*labels)  # Tách chẩn đoán mắt trái và mắt phải

# Tạo bảng hiển thị các cặp chẩn đoán phổ biến
fig = go.Figure(data=[go.Table(
    header=dict(values=['Chẩn đoán mắt trái', 'Chẩn đoán mắt phải', 'Số lượng']),
    cells=dict(values=[left_labels, right_labels, counts])  # Dữ liệu cho các ô trong bảng
)])

fig.update_layout(title='Các cặp chẩn đoán mắt trái và mắt phải phổ biến nhất')  # Tiêu đề bảng
fig.show()  # Hiển thị bảng

# Ghi chú: Mỗi cặp chẩn đoán phải xuất hiện với nhau ít nhất một lần

In [ ]:
# Hiển thị phân bố độ tuổi của bệnh nhân
fig = px.histogram(
    df,
    x='Patient Age',  # Trục x là độ tuổi bệnh nhân
    nbins=30,  # Số lượng bins trong biểu đồ histogram
    title='Phân bố độ tuổi'  # Tiêu đề biểu đồ
)
# Ghi chú: Độ tuổi tập trung nhiều nhất từ 60 đến 64
fig.show()  # Hiển thị biểu đồ histogram

# Tạo biểu đồ scatter để xem mối quan hệ giữa độ tuổi và giới tính bệnh nhân
fig = px.scatter(
    df,
    x='Patient Age',  # Trục x là độ tuổi bệnh nhân
    y='Patient Sex',  # Trục y là giới tính bệnh nhân
    color='Left-Diagnostic Keywords',  # Màu sắc dựa trên chẩn đoán mắt trái
    title='Độ tuổi so với giới tính theo chẩn đoán'  # Tiêu đề biểu đồ
)
fig.show()  # Hiển thị biểu đồ scatter

# Tạo biểu đồ scatter cho mối quan hệ giữa độ tuổi và giới tính bệnh nhân với chẩn đoán mắt phải
fig = px.scatter(
    df,
    x='Patient Age',  # Trục x là độ tuổi bệnh nhân
    y='Patient Sex',  # Trục y là giới tính bệnh nhân
    color='Right-Diagnostic Keywords',  # Màu sắc dựa trên chẩn đoán mắt phải
    title='Độ tuổi so với giới tính theo chẩn đoán'  # Tiêu đề biểu đồ
)
fig.show()  # Hiển thị biểu đồ scatter

In [ ]:
# Hàm này để lấy kích thước của các hình ảnh trong thư mục
def get_image_sizes(image_folder):
    sizes = []  # Danh sách lưu trữ kích thước hình ảnh
    for file_name in os.listdir(image_folder):
        if file_name.endswith(('.jpg', '.jpeg', '.png')):  # Kiểm tra định dạng tệp
            with Image.open(os.path.join(image_folder, file_name)) as img:
                sizes.append(img.size)  # Thêm kích thước vào danh sách
    return sizes

# Lấy kích thước từ hình ảnh huấn luyện
image_sizes = get_image_sizes('/kaggle/input/nhn-bit-bnh-v-mt/Ocular Disease Recognition/ODIR-5K/ODIR-5K/Testing Images')
sizes_df = pd.DataFrame(image_sizes, columns=['Width', 'Height'])  # Tạo DataFrame từ kích thước

# Vẽ phân bố kích thước hình ảnh
fig = px.histogram(sizes_df, x='Width', nbins=30, title='Phân bố chiều rộng hình ảnh')
# Thêm biểu đồ cho chiều cao hình ảnh
fig.add_trace(px.histogram(sizes_df, x='Height', nbins=30).data[0])
fig.update_layout(xaxis_title='Giá trị pixel', yaxis_title='Tần suất', barmode='overlay')  # Cập nhật nhãn trục
fig.show()  # Hiển thị biểu đồ

# Ghi chú: Hình ảnh có kích thước khác nhau

In [ ]:
import random

# Hàm này để lấy phân bố màu từ một mẫu hình ảnh
def get_sampled_color_distribution(image_folder, num_images=100, pixels_per_image=1000):
    colors = []  # Danh sách lưu trữ màu
    image_files = [f for f in os.listdir(image_folder) if f.endswith(('.jpg', '.jpeg', '.png'))]
    sampled_files = random.sample(image_files, min(num_images, len(image_files)))  # Chọn mẫu hình ảnh

    for file_name in sampled_files:
        with Image.open(os.path.join(image_folder, file_name)) as img:
            img = img.convert('RGB')  # Chuyển đổi hình ảnh sang RGB
            np_image = np.array(img)
            # Làm phẳng hình ảnh thành 2D và lấy mẫu pixel
            sampled_pixels = np_image.reshape(-1, 3)
            sampled_pixels = sampled_pixels[random.sample(range(sampled_pixels.shape[0]), min(pixels_per_image, sampled_pixels.shape[0]))]
            colors.extend(sampled_pixels)  # Thêm màu vào danh sách

    return np.array(colors)  # Trả về mảng màu

# Lấy phân bố màu từ một mẫu hình ảnh
sampled_colors = get_sampled_color_distribution('/kaggle/input/nhn-bit-bnh-v-mt/Ocular Disease Recognition/ODIR-5K/ODIR-5K/Training Images')

mean_colors = np.mean(sampled_colors, axis=0)  # Tính toán trung bình màu
std_colors = np.std(sampled_colors, axis=0)    # Tính toán độ lệch chuẩn màu

# Vẽ phân bố màu
fig = go.Figure()
fig.add_trace(go.Bar(x=['Đỏ', 'Xanh lá', 'Xanh dương'], y=mean_colors, name='Cường độ màu trung bình'))
fig.add_trace(go.Bar(x=['Đỏ', 'Xanh lá', 'Xanh dương'], y=std_colors, name='Độ lệch chuẩn cường độ màu', opacity=0.6))
fig.update_layout(yaxis_title='Cường độ màu', title='Phân bố màu của các hình ảnh mẫu')  # Cập nhật tiêu đề
fig.show()  # Hiển thị biểu đồ

In [ ]:
# Ma trận tương quan giữa tuổi và chẩn đoán cả hai mắt
# Tạo bản sao của DataFrame để mã hóa các cột chẩn đoán
encoded_df = df.copy()

# Mã hóa các cột chẩn đoán
encoded_df['Left-Diagnostic Keywords'] = pd.factorize(encoded_df['Left-Diagnostic Keywords'])[0]
encoded_df['Right-Diagnostic Keywords'] = pd.factorize(encoded_df['Right-Diagnostic Keywords'])[0]
# Sử dụng factorize để chuyển đổi giá trị phân loại thành giá trị số, giúp xử lý và vẽ biểu đồ

# Tính toán ma trận tương quan
correlation_matrix = encoded_df[['Patient Age', 'Left-Diagnostic Keywords', 'Right-Diagnostic Keywords']].corr()

# Vẽ heatmap cho ma trận tương quan
fig = px.imshow(correlation_matrix, text_auto=True, color_continuous_scale='RdBu_r',
                title='Heatmap Tương Quan Giữa Độ Tuổi và Các Chẩn Đoán')
fig.show()  # Hiển thị heatmap

In [ ]:
plt.figure(figsize=(8,8))
plt.pie(df['Patient Sex'].value_counts(),labels=["Nam","Nữ"],autopct="%0.1f%%",shadow=True,explode=[0.2,0])

In [ ]:
# Lấy 5 chẩn đoán phổ biến nhất cho mắt trái và mắt phải
top_left_diagnoses = df['Left-Diagnostic Keywords'].value_counts().nlargest(5).index
top_right_diagnoses = df['Right-Diagnostic Keywords'].value_counts().nlargest(5).index

# Lọc DataFrame để chỉ bao gồm các chẩn đoán này
filtered_df = df[(df['Left-Diagnostic Keywords'].isin(top_left_diagnoses)) &
                  (df['Right-Diagnostic Keywords'].isin(top_right_diagnoses))]

# Chuẩn bị dữ liệu cho biểu đồ Sankey
left_labels = filtered_df['Left-Diagnostic Keywords'].unique().tolist()
right_labels = filtered_df['Right-Diagnostic Keywords'].unique().tolist()

labels = left_labels + right_labels
source = filtered_df['Left-Diagnostic Keywords'].apply(lambda x: labels.index(x)).tolist()
target = filtered_df['Right-Diagnostic Keywords'].apply(lambda x: labels.index(x) + len(left_labels)).tolist()
values = [1] * len(filtered_df)  # Mỗi dòng đại diện cho 1 ví dụ

# Tạo biểu đồ Sankey
fig = go.Figure(go.Sankey(
    node=dict(
        pad=15,
        thickness=20,
        line=dict(color="black", width=0.2),
        label=labels
    ),
    link=dict(
        source=source,
        target=target,
        value=values
    )
))

fig.update_layout(title_text="Biểu đồ Sankey Đơn Giản về Dòng Chảy Chẩn Đoán Giữa Mắt Trái và Mắt Phải", font_size=10)
fig.show()  # Hiển thị biểu đồ Sankey

In [ ]:
# filtered_df thường được sử dụng để chỉ một DataFrame đã được lọc theo 
# một tiêu chí nhất định trong pandas. Dưới đây là giải thích chi tiết về khái niệm này.

filtered_df

In [ ]:
import plotly.express as px

# Tạo biểu đồ phân tán với hiệu ứng hoạt hình
fig = px.scatter(
    filtered_df,
    x="Left-Diagnostic Keywords",  # Trục x: Chẩn đoán mắt trái
    y="Right-Diagnostic Keywords",  # Trục y: Chẩn đoán mắt phải
    color="Patient Sex",            # Màu sắc: Giới tính bệnh nhân
    animation_frame="Patient Age",  # Khung hình hoạt hình: Độ tuổi bệnh nhân
    template="plotly_dark",         # Giao diện tối
    title="Animation of Diagnoses between Left and Right Eyes by Patient Age",  # Tiêu đề
    labels={
        "Left-Diagnostic Keywords": "Chẩn Đoán Mắt Trái",  # Nhãn cho trục x
        "Right-Diagnostic Keywords": "Chẩn Đoán Mắt Phải",  # Nhãn cho trục y
        "Patient Sex": "Giới Tính Bệnh Nhân",              # Nhãn cho màu sắc
        "Patient Age": "Độ Tuổi Bệnh Nhân"                  # Nhãn cho khung hình hoạt hình
    }
)

fig.show()  # Hiển thị biểu đồ

# Chuẩn bị và xử lý dữ liệu

In [ ]:
# Hàm tùy chỉnh để phân loại chẩn đoán bệnh
def map_to_label_exact(keyword):
    if keyword == 'normal fundus':
        return 'Normal Fundus'  # Chẩn đoán bình thường
    elif keyword == 'retinopathy':
        return 'Diabetes'  # Bệnh võng mạc
    elif keyword == 'severe nonproliferative retinopathy':
        return 'Diabetes'  # Bệnh võng mạc nặng
    elif keyword == 'cataract':
        return 'Cataract'  # Đục thủy tinh thể
    elif keyword == 'moderate non proliferative retinopathy':
        return 'Diabetes'  # Bệnh võng mạc vừa
    elif keyword == 'pathological myopia':
        return 'Pathological Myopia'  # Cận thị bệnh lý
    elif keyword == 'dry age-related macular degeneration':
        return 'Age-related Macular Degeneration'  # Thoái hóa điểm vàng do tuổi tác
    elif keyword == 'glaucoma':
        return 'Glaucoma'  # Glaucom
    elif keyword == 'drusen':
        return 'Drusen'  # Nốt Drusen
    elif keyword == 'hypertensive retinopathy':
        return 'Hypertension'  # Bệnh võng mạc do tăng huyết áp
    else:
        return 'Other diseases/abnormalities'  # Các bệnh hoặc bất thường khác

In [ ]:
# Áp dụng hàm tùy chỉnh để phân loại chẩn đoán cho mắt phải và mắt trái
df['right_label_column'] = df['Right-Diagnostic Keywords'].apply(map_to_label_exact)  # Phân loại chẩn đoán mắt phải
df['left_label_column'] = df['Left-Diagnostic Keywords'].apply(map_to_label_exact)    # Phân loại chẩn đoán mắt trái

In [ ]:
# Tạo DataFrame cho chẩn đoán mắt phải
df_right_img = df[['Right-Diagnostic Keywords', 'right_label_column']]  # Chọn cột chẩn đoán mắt phải và nhãn tương ứng

# Tạo DataFrame cho chẩn đoán mắt trái
df_left_img = df[['Left-Diagnostic Keywords', 'left_label_column']]  # Chọn cột chẩn đoán mắt trái và nhãn tương ứng

In [ ]:
df_right_img  # Hiển thị các hàng của DataFrame df_right_img

In [ ]:
# Kết hợp DataFrame df_right_img với cột "Right-Fundus" từ df
df_right_img = pd.concat([df_right_img, df[['Right-Fundus']]], axis=1)  
df_right_img.rename(columns={"right_label_column": "disease", "Right-Fundus": "image_id"}, inplace=True)  # Đổi tên cột
df_right_img.drop("Right-Diagnostic Keywords", axis=1, inplace=True)  # Xóa cột tên chẩn đoán mắt phải

# Thực hiện tương tự cho mắt trái
df_left_img = pd.concat([df_left_img, df[['Left-Fundus']]], axis=1)  
df_left_img.rename(columns={"left_label_column": "disease", "Left-Fundus": "image_id"}, inplace=True)

# Kết hợp cả hai DataFrame vào một DataFrame duy nhất
df_labeled = pd.concat([df_right_img, df_left_img])  
df_labeled.drop("Left-Diagnostic Keywords", axis=1, inplace=True)  # Xóa cột tên chẩn đoán mắt trái

# Đếm số lượng các bệnh trong cột 'disease'
df_labeled['disease'].value_counts()

In [ ]:
df_labeled # Hiển thị các hàng của DataFrame df_labeled

In [ ]:
# Giữ lại chỉ các loại bệnh cụ thể, loại bỏ các loại bệnh không rõ ràng
df_labeled = df_labeled.loc[
    (df_labeled["disease"] == "Normal Fundus") | 
    (df_labeled['disease'] == 'Cataract') | 
    (df_labeled['disease'] == 'Diabetes') | 
    (df_labeled['disease'] == 'Pathological Myopia') | 
    (df_labeled['disease'] == 'Glaucoma') | 
    (df_labeled['disease'] == 'Age-related Macular Degeneration') | 
    (df_labeled['disease'] == 'Drusen') | 
    (df_labeled['disease'] == 'Hypertension')
]

In [ ]:
df_labeled['disease'].value_counts()  # Đếm số lượng các loại bệnh trong DataFrame df_labeled

In [ ]:
import shutil

# Đường dẫn chính cho thư mục chứa ảnh đã phân loại
main_dir = 'classified_images_new/train'
os.makedirs(main_dir, exist_ok=True)  # Tạo thư mục chính nếu chưa tồn tại

# Tạo các thư mục con cho từng loại bệnh
labels = df_labeled['disease'].unique()  # Lấy danh sách các nhãn duy nhất
for label in labels:
    os.makedirs(os.path.join(main_dir, label), exist_ok=True)  # Tạo thư mục cho từng nhãn

# Đường dẫn chứa ảnh
place_imgs = "/kaggle/input/nhn-bit-bnh-v-mt/Ocular Disease Recognition/preprocessed_images"

# Sao chép ảnh vào các thư mục tương ứng
for index, row in df_labeled.iterrows():
    place_file = os.path.join(place_imgs, row["image_id"])  # Đường dẫn đến ảnh gốc
    label = row["disease"]  # Lấy nhãn bệnh
    im_folder = os.path.join(main_dir, label)  # Đường dẫn đến thư mục bệnh cụ thể
    im_files = os.path.join(im_folder, row["image_id"])  # Đường dẫn đến ảnh sẽ sao chép

    # Kiểm tra xem file có tồn tại không
    if not os.path.isfile(place_file):
        print(f"File {place_file} does not exist.")  # In ra thông báo nếu file không tồn tại
        continue

    shutil.copy2(place_file, im_files)  # Sao chép file vào thư mục tương ứng

In [ ]:
import random

train_dir = './classified_images_new/train'  # Đường dẫn đến thư mục train
test_dir = './classified_images_new/test'    # Đường dẫn đến thư mục test

# Tạo thư mục test nếu chưa tồn tại
os.makedirs(test_dir, exist_ok=True)

# Kích thước test là 20%
test_size = 0.2

# Lặp qua từng thư mục nhãn trong thư mục train
for label_folder in os.listdir(train_dir):
    label_train_dir = os.path.join(train_dir, label_folder)  # Đường dẫn đến thư mục nhãn trong train
    label_test_dir = os.path.join(test_dir, label_folder)    # Đường dẫn đến thư mục nhãn trong test

    os.makedirs(label_test_dir, exist_ok=True)  # Tạo thư mục cho nhãn trong test

    images = os.listdir(label_train_dir)  # Lấy danh sách ảnh trong thư mục train

    # Xáo trộn danh sách ảnh
    random.shuffle(images)

    num_test_images = int(len(images) * test_size)  # Tính số lượng ảnh test

    test_images = images[:num_test_images]  # Lấy danh sách ảnh test

    # Di chuyển ảnh test vào thư mục test
    for image in test_images:
        src_path = os.path.join(label_train_dir, image)  # Đường dẫn gốc của ảnh
        dst_path = os.path.join(label_test_dir, image)   # Đường dẫn đích của ảnh
        shutil.move(src_path, dst_path)  # Di chuyển ảnh

print("Test set created successfully.")  # In ra thông báo thành công

In [ ]:
train_dir = './classified_images_new/train'  # Đường dẫn đến thư mục train
valid_dir = './classified_images_new/valid'  # Đường dẫn đến thư mục valid

# Tạo thư mục valid nếu chưa tồn tại
os.makedirs(valid_dir, exist_ok=True)

# Kích thước valid là 10%
valid_size = 0.1

# Lặp qua từng thư mục nhãn trong thư mục train
for label_folder in os.listdir(train_dir):
    label_train_dir = os.path.join(train_dir, label_folder)  # Đường dẫn đến thư mục nhãn trong train
    label_valid_dir = os.path.join(valid_dir, label_folder)  # Đường dẫn đến thư mục nhãn trong valid

    os.makedirs(label_valid_dir, exist_ok=True)  # Tạo thư mục cho nhãn trong valid

    images = os.listdir(label_train_dir)  # Lấy danh sách ảnh trong thư mục train

    # Xáo trộn danh sách ảnh
    random.shuffle(images)

    num_valid_images = int(len(images) * valid_size)  # Tính số lượng ảnh valid

    valid_images = images[:num_valid_images]  # Lấy danh sách ảnh valid

    # Di chuyển ảnh valid vào thư mục valid
    for image in valid_images:
        src_path = os.path.join(label_train_dir, image)  # Đường dẫn gốc của ảnh
        dst_path = os.path.join(label_valid_dir, image)   # Đường dẫn đích của ảnh
        shutil.move(src_path, dst_path)  # Di chuyển ảnh

print("Validation set created successfully.")  # In ra thông báo thành công

# Tiền xử lý dữ liệu và tăng cường dữ liệu

In [ ]:
# Đường dẫn đến thư mục train
train_dir = "/kaggle/working/classified_images_new/train"

# Đếm số lượng ảnh trong mỗi thư mục nhãn
class_counts_train = {folder: len(os.listdir(os.path.join(train_dir, folder))) for folder in os.listdir(train_dir)}

print("Class counts:") # Số lớp
for label, count in class_counts_train.items():
    # In ra số lượng ảnh cho mỗi nhãn
    print(f"{label}: {count} images")
    # Kiểm tra nếu dữ liệu không cân bằng
    if count < max(class_counts_train.values()):  # Nếu số lượng ảnh nhỏ hơn số lượng lớn nhất
        print("100% dữ liệu không được cân bằng.")

In [ ]:
# Đường dẫn đến thư mục test
test_dir = "/kaggle/working/classified_images_new/test"

# Đếm số lượng ảnh trong mỗi thư mục nhãn
class_counts_test = {folder: len(os.listdir(os.path.join(test_dir, folder))) for folder in os.listdir(test_dir)}

print("Class counts:") # Số lớp
for label, count in class_counts_test.items():
    # In ra số lượng ảnh cho mỗi nhãn trong tập test
    print(f"{label}: {count} images")
    if count < max(class_counts_test.values()):  # Nếu số lượng ảnh nhỏ hơn số lượng lớn nhất
        print("100% dữ liệu không được cân bằng.")

In [ ]:
# Đường dẫn đến thư mục valid
valid_dir = "/kaggle/working/classified_images_new/valid"

# Đếm số lượng ảnh trong mỗi thư mục nhãn
class_counts_valid = {folder: len(os.listdir(os.path.join(valid_dir, folder))) for folder in os.listdir(valid_dir)}

print("Class counts:")
for label, count in class_counts_valid.items():
    # In ra số lượng ảnh cho mỗi nhãn trong tập valid
    print(f"{label}: {count} images")

In [ ]:
# Định nghĩa số lượng mục tiêu cho mỗi lớp
max_count = max(class_counts_train.values())

for label, count in class_counts_train.items():
    if count < max_count:
        # Đường dẫn đến thư mục nhãn
        label_folder = os.path.join(train_dir, label)
        images = os.listdir(label_folder)
        
        # Nhân đôi ảnh để đạt số lượng mục tiêu
        while len(images) < max_count:
            images.extend(images[:max_count - len(images)])  # Sao chép ảnh cho đến khi đủ
        
        # Xáo trộn danh sách ảnh
        np.random.shuffle(images)
        
        # Sao chép ảnh vào thư mục nhãn
        for image in images[count:]:
            src = os.path.join(label_folder, image)  # Đường dẫn gốc của ảnh
            dst = os.path.join(label_folder, f"duplicated_{image}")  # Đường dẫn đích với tên mới
            shutil.copy2(src, dst)  # Sao chép ảnh

In [ ]:
# In ra số lượng ảnh trong từng lớp
print("Class counts:") # Số lượng lớp
for label, count in class_counts_train.items():
    # In ra nhãn và số lượng ảnh tương ứng
    print(f"{label}: {count} images")

In [ ]:
# Định nghĩa kích thước mục tiêu
target_size = (224, 224)

# Hàm thay đổi kích thước ảnh
def resize_image(image_path, target_path):
    with Image.open(image_path) as img:  # Mở ảnh
        img = img.resize(target_size)  # Thay đổi kích thước ảnh
        img.save(target_path)  # Lưu ảnh đã thay đổi kích thước

# Thay đổi kích thước ảnh trong từng thư mục
for label in class_counts_train.keys():
    label_folder = os.path.join(train_dir, label)  # Đường dẫn đến thư mục nhãn
    for image_file in os.listdir(label_folder):  # Lặp qua từng ảnh trong thư mục
        src_path = os.path.join(label_folder, image_file)  # Đường dẫn gốc của ảnh
        dst_path = os.path.join(label_folder, image_file)  # Ghi đè lên tệp gốc
        resize_image(src_path, dst_path)  # Thay đổi kích thước ảnh

In [ ]:
# Định nghĩa kích thước mục tiêu
target_size = (224, 224)

# Hàm thay đổi kích thước ảnh
def resize_image(image_path, target_path):
    with Image.open(image_path) as img:  # Mở ảnh
        img = img.resize(target_size)  # Thay đổi kích thước ảnh
        img.save(target_path)  # Lưu ảnh đã thay đổi kích thước

# Thay đổi kích thước ảnh trong từng thư mục
for label in class_counts_valid.keys():
    label_folder = os.path.join(valid_dir, label)  # Đường dẫn đến thư mục nhãn
    for image_file in os.listdir(label_folder):  # Lặp qua từng ảnh trong thư mục
        src_path = os.path.join(label_folder, image_file)  # Đường dẫn gốc của ảnh
        dst_path = os.path.join(label_folder, image_file)  # Ghi đè lên tệp gốc
        resize_image(src_path, dst_path)  # Thay đổi kích thước ảnh

In [ ]:
# Định nghĩa kích thước mục tiêu
target_size = (224, 224)

# Hàm thay đổi kích thước ảnh
def resize_image(image_path, target_path):
    with Image.open(image_path) as img:  # Mở ảnh
        img = img.resize(target_size)  # Thay đổi kích thước ảnh
        img.save(target_path)  # Lưu ảnh đã thay đổi kích thước

# Thay đổi kích thước ảnh trong từng thư mục
for label in class_counts_test.keys():
    label_folder = os.path.join(test_dir, label)  # Đường dẫn đến thư mục nhãn
    for image_file in os.listdir(label_folder):  # Lặp qua từng ảnh trong thư mục
        src_path = os.path.join(label_folder, image_file)  # Đường dẫn gốc của ảnh
        dst_path = os.path.join(label_folder, image_file)  # Ghi đè lên tệp gốc
        resize_image(src_path, dst_path)  # Thay đổi kích thước ảnh

In [ ]:
# Hàm chuẩn hóa ảnh
def normalize_image(image):
    # Chuyển đổi ảnh về kiểu float32 và chuẩn hóa về khoảng [0, 1]
    image = image.astype('float32') / 255.0
    return image

# Hàm xử lý tất cả ảnh trong thư mục
def process_images_in_directory(directory):
    for filename in os.listdir(directory):  # Lặp qua từng tệp trong thư mục
        file_path = os.path.join(directory, filename)  # Đường dẫn đầy đủ của tệp
        if os.path.isfile(file_path):  # Kiểm tra nếu đây là tệp
            # Tải ảnh ở chế độ xám
            img = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)
            # Chuẩn hóa ảnh
            img_normalized = normalize_image(img)
            # Lưu ảnh đã chuẩn hóa trở lại tệp gốc
            cv2.imwrite(file_path, (img_normalized * 255).astype('uint8'))

# Xử lý ảnh trong các thư mục train, valid và test
process_images_in_directory(train_dir)
process_images_in_directory(valid_dir)
process_images_in_directory(test_dir)

In [ ]:
# Tạo đối tượng ImageDataGenerator cho tập huấn luyện với tăng cường dữ liệu
train_datagen = ImageDataGenerator(  # Tăng cường dữ liệu
    rescale=1./255,  # Chuẩn hóa pixel về khoảng [0, 1]
    rotation_range=20,  # Xoay ảnh ngẫu nhiên trong khoảng 20 độ
    width_shift_range=0.2,  # Dịch chuyển ảnh theo chiều ngang (20% chiều rộng)
    height_shift_range=0.2,  # Dịch chuyển ảnh theo chiều dọc (20% chiều cao)
    shear_range=0.2,  # Tăng cường độ cắt (shear) ngẫu nhiên
    zoom_range=0.2,  # Phóng to ảnh ngẫu nhiên (20%)
    horizontal_flip=True,  # Lật ngang ảnh ngẫu nhiên
    fill_mode='nearest'  # Phương pháp điền cho các pixel mới
)

# Tạo đối tượng ImageDataGenerator cho tập valid mà không có tăng cường
valid_datagen = ImageDataGenerator(rescale=1./255)  # Chỉ chuẩn hóa

# Tạo đối tượng ImageDataGenerator cho tập test mà không có tăng cường
test_datagen = ImageDataGenerator(rescale=1./255)  # Chỉ chuẩn hóa

In [ ]:
# Tạo bộ dữ liệu cho tập huấn luyện từ thư mục
train_generator = train_datagen.flow_from_directory(
    '/kaggle/working/classified_images_new/train',  # Đường dẫn đến thư mục huấn luyện
    target_size=(224, 224),  # Kích thước ảnh đầu ra
    batch_size=32,  # Số lượng ảnh trong mỗi lô
    class_mode='categorical'  # Kiểu nhãn cho đa lớp
)

# Tạo bộ dữ liệu cho tập xác thực từ thư mục
valid_generator = valid_datagen.flow_from_directory(
    '/kaggle/working/classified_images_new/valid',  # Đường dẫn đến thư mục xác thực
    target_size=(224, 224),  # Kích thước ảnh đầu ra
    batch_size=32,  # Số lượng ảnh trong mỗi lô
    class_mode='categorical'  # Kiểu nhãn cho đa lớp
)

# Tạo bộ dữ liệu cho tập kiểm tra từ thư mục
test_generator = test_datagen.flow_from_directory(
    '/kaggle/working/classified_images_new/test',  # Đường dẫn đến thư mục kiểm tra
    target_size=(224, 224),  # Kích thước ảnh đầu ra
    batch_size=32,  # Số lượng ảnh trong mỗi lô
    class_mode='categorical',  # Kiểu nhãn cho đa lớp
    shuffle=False  # Không xáo trộn cho tập kiểm tra
)

# Tinh chỉnh mô hình được đào tạo trước (EfficientNetB0)

In [ ]:
from tensorflow.keras import layers, models, regularizers

# Tải EfficientNetB0 với trọng số từ ImageNet, không bao gồm lớp đầu ra
eff = EfficientNetB0(weights="imagenet", include_top=False, input_shape=(224, 224, 3))

# Thiết lập tất cả các tham số là có thể huấn luyện
for layer in eff.layers:
    layer.trainable = True

# Lấy số lượng đầu vào cho lớp fully connected (fc)
n_inputs = eff.output_shape[1]

# Định nghĩa lại lớp fc cho bài toán phân loại
eff_top = models.Sequential([
    # Convolutional Block 1
    layers.Conv2D(32, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l2(0.002)),
    layers.BatchNormalization(),  # Thêm Batch Normalization
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.5),
    
    # Convolutional Block 2
    layers.Conv2D(64, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l2(0.002)),
    layers.Conv2D(128, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l2(0.002)),
    layers.BatchNormalization(),  # Thêm Batch Normalization
    layers.GlobalAveragePooling2D(),  # Sử dụng Global Average Pooling
    layers.Dropout(0.5),
    
    # Fully Connected Layer
    layers.Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.002)),  # Giảm số đơn vị
    layers.Dropout(0.6),  # Tăng tỷ lệ dropout
    
    # Output Layer
    layers.Dense(8, activation='softmax')  # 8 lớp đầu ra cho phân loại
])

# Kết hợp mô hình cơ sở EfficientNet với các lớp mới
model = models.Sequential([
    eff,
    eff_top
])

# In tóm tắt mô hình
model.summary()

In [ ]:
from tensorflow.keras import layers, models, regularizers

# Tải EfficientNetB0 với trọng số từ ImageNet, không bao gồm lớp đầu ra
eff = EfficientNetB0(weights="imagenet", include_top=False, input_shape=(224, 224, 3))

# Thiết lập tất cả các tham số là có thể huấn luyện
for layer in eff.layers:
    layer.trainable = True

# Lấy số lượng đầu vào cho lớp fully connected (fc)
n_inputs = eff.output_shape[1]

# Định nghĩa lại lớp fc cho bài toán phân loại
eff_top = models.Sequential([
    layers.GlobalAveragePooling2D(), # Add this line to remove spatial dimensions
    layers.BatchNormalization(),  # Thêm Batch Normalization
    layers.Dropout(0.5),
    
    # Fully Connected Layer
    layers.Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.002)),  # Giảm số đơn vị
    layers.Dropout(0.6),  # Tăng tỷ lệ dropout
    
    # Output Layer
    layers.Dense(8, activation='softmax')  # 8 lớp đầu ra cho phân loại
])

# Kết hợp mô hình cơ sở EfficientNet với các lớp mới
model = models.Sequential([
    eff,
    eff_top
])

# Build the model to define the output shapes of all layers
model.build(input_shape=(None, 224, 224, 3))

# In tóm tắt mô hình
model.summary()

In [ ]:
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.losses import SparseCategoricalCrossentropy

# Biên dịch mô hình
model.compile(
    optimizer=Adam(learning_rate=3e-4),  # Sử dụng Adam với tốc độ học 3e-4
    loss="categorical_crossentropy",  # Sử dụng hàm mất mát categorical_crossentropy
    metrics=['accuracy']  # Đo lường độ chính xác
)

In [ ]:
# Thiết lập callback để dừng sớm
early_stopping_callback = EarlyStopping(
    monitor='loss',   # Theo dõi mất mát trong quá trình huấn luyện
    patience=10,      # Số epoch không cải thiện sau đó huấn luyện sẽ dừng lại
    restore_best_weights=True  # Khôi phục trọng số của mô hình về epoch tốt nhất
)

# Thiết lập callback để lưu mô hình tốt nhất
checkpoint_callback = ModelCheckpoint(
    filepath='best_model.h5',  # Đường dẫn lưu mô hình
    monitor='val_accuracy',        # Theo dõi độ chính xác xác thực
    save_best_only=True,          # Chỉ lưu mô hình có độ chính xác tốt nhất
    save_weights_only=False,       
    mode='max'                    # Lưu khi độ chính xác cao nhất
)       

# Huấn luyện mô hình với dữ liệu tăng cường và trọng số lớp
history = model.fit(
    train_generator,
    steps_per_epoch=(train_generator.samples // train_generator.batch_size),  # Số bước mỗi epoch
    epochs=200,  # Số lượng epoch lớn để đảm bảo huấn luyện đầy đủ
    validation_data=valid_generator,  # Dữ liệu xác thực
    callbacks=[early_stopping_callback, checkpoint_callback],  # Sử dụng các callback đã thiết lập
    verbose=0  # Không in thông tin huấn luyện
)

In [ ]:
import numpy as np  # Nhập thư viện numpy

# Dự đoán cho tập kiểm tra
y_pred = []  # Khởi tạo danh sách để lưu trữ dự đoán
for i in model.predict(test_generator):  # Dự đoán cho từng mẫu trong test_generator
    y_pred.append(np.argmax(np.array(i)).astype("int32"))  # Lấy nhãn có xác suất cao nhất và chuyển đổi thành int32

In [ ]:
# Hàm để lấy nhãn từ chỉ số
def get_label_from_index(index):
    labels = ["Normal", "Cataract", "Diabetes", "Glaucoma", "Hypertension", "Myopia", "Age Issues", "Other"]
    return labels[index]

# Lấy một lô ảnh và nhãn từ test_generator
x_batch, y_batch = next(test_generator)

# Dự đoán lớp cho lô ảnh
y_pred = model.predict(x_batch)
y_pred_classes = np.argmax(y_pred, axis=1)  # Lấy chỉ số lớp dự đoán

# Thiết lập hình ảnh để hiển thị
plt.figure(figsize=(12, 7))

for i in range(20):
    # Chọn một chỉ số ngẫu nhiên trong lô
    idx = random.choice(range(len(x_batch)))
    
    # Lấy ảnh và nhãn thực tế cùng nhãn dự đoán
    image = x_batch[idx]
    true_label = np.argmax(y_batch[idx])  # Nhãn thực tế
    pred_label = y_pred_classes[idx]  # Nhãn dự đoán
    
    # Chuyển đổi chỉ số thành tên lớp
    true_label_name = get_label_from_index(true_label)
    pred_label_name = get_label_from_index(pred_label)
    
    # Hiển thị ảnh và nhãn
    plt.subplot(4, 5, i + 1)  # Tạo lưới hiển thị
    plt.imshow(image)  # Hiển thị ảnh
    plt.xlabel(f"Actual: {true_label_name}\nPrediction: {pred_label_name}")  # Nhãn thực tế và dự đoán

plt.tight_layout()  # Điều chỉnh khoảng cách giữa các ảnh
plt.show()  # Hiển thị các ảnh

In [ ]:
# Hàm thu thập nhãn thực tế và dự đoán
def collect_labels_and_predictions(generator, model):
    true_labels = []
    pred_labels = []
    
    for batch_images, batch_labels in generator:
        # Dự đoán cho từng lô ảnh
        batch_preds = model.predict(batch_images)
        # Lấy lớp dự đoán
        batch_pred_classes = np.argmax(batch_preds, axis=1)
        # Lấy lớp thực tế
        batch_true_classes = np.argmax(batch_labels, axis=1)
        
        # Thêm vào danh sách
        pred_labels.extend(batch_pred_classes)
        true_labels.extend(batch_true_classes)
        
        # Dừng lại nếu đã lấy đủ mẫu
        if len(true_labels) >= generator.samples:
            break

    return np.array(true_labels), np.array(pred_labels)

# Đánh giá mô hình bằng cách sử dụng generator
loss, accuracy = model.evaluate(test_generator, steps=test_generator.samples // test_generator.batch_size)
print("Loss:", loss)
print("Accuracy:", accuracy)

# Thu thập nhãn thực tế và dự đoán
y_true_classes, y_pred_classes = collect_labels_and_predictions(test_generator, model)

# Danh sách tên lớp
class_names = ['Normal', 'Cataract', 'Diabetes', 'Glaucoma', 'Hypertension', 'Myopia', 'Age Issues', 'Other']

# Tạo và in báo cáo phân loại
report = classification_report(y_true_classes, y_pred_classes, target_names=class_names)
print(report)

# Tạo và vẽ ma trận nhầm lẫn
cm = confusion_matrix(y_true_classes, y_pred_classes)
plt.figure(figsize=(8, 8))
sns.heatmap(cm, annot=True, fmt="d", cmap="Greens", xticklabels=class_names, yticklabels=class_names)
plt.xlabel("Predicted Class")
plt.ylabel("True Class")
plt.title("Confusion Matrix")
plt.show()

# In tổng số mẫu cho mỗi lớp
class_counts = {name: np.sum(y_true_classes == idx) for idx, name in enumerate(class_names)}

print("Total samples per class:")
for class_name, count in class_counts.items():
    print(f"{class_name}: {count}")    

In [ ]:
model.save('my_model.h5')